### Calculate the probability of zero precipitation (proportion of dry days) for all stations within 20 km of FR: FR, UC, HM, K, BV

It was important to characterise the statistics of precipitation in Marmot Creek Research Basin (MCRB). Python code was written to read and analyse total daily precipitation data, i.e. rainfall and snowfall, initially from three weather stations at Fisera Ridge (2325 m a.s.l.), Upper Clearing (1845 m a.s.l.) and Hay Meadow (1436 m a.s.l.). Snowfall amounts were based on snow water equivalent (SWE) values. It became evident that, to improve the quality of the spatiotemporal precipitation fields generated using the Random Mixing method, it would be preferable to access precipitation data from other gauges in the vicinity. Additional stations within an initial 50km radius of MCRB were identified from https://climate.weather.gc.ca/historical_data/search_historic_data_e.html. Several of these had sparse data for the priod of interest: however, five gauges were identified that had recorded daily data over the same period as those at MCRB, i.e. October 2005 to September 2016. These are at: Kananaskis Pocaterra (1610.0 m a.s.l.), Banff CS (1397 m a.s.l.), Kananaskis (1391 m a.s.l.), Bow Valley (1298 m a.s.l.) and Wildcat Hills (1268 m a.s.l.). There were some periods of missing data, as highlighted in Table *** (put in a table with missing dates for each of the new stations). Python code was adapted to deal with missing data, simply by assigning NaN to those dates.

Once all the relevant data was read in, the first step in assessing precipitation statistics was to calculate the probability of experiencing a dry day, using four?!?!? different dry day threshold values: 0 mm, less than 0.1 mm, less than or equal to 0.1 mm and less than 1.0 mm. ?!?!? _Need to provide some rationale for why these values were chosen?!?!?_. This resulted in a set of dry day probability values which is shown in Table ?!?!? (_see the table at C:\Users\b1043453\OneDrive - Newcastle University\OnePlanet PhD\Random_mixing\RMWSPy_Horning_and_Haese_2021\RMWSPy-master\MCRB_examples\MCRB_gauges_only\Characterising_P_statistics\Prob_zero_P_2005-2016.csv)_. The chance of a dry day is exactly the same for the less than 0.1 mm and less than or equal to 0.1 mm thresholds. Of these two, only the less than 0.1 mm limit has been retained from this point onwards. In addition, due to tolerance limits of precipitation gauges, it is generally poor prectice to adopt a dry day threshold of exactly 0 mm, hence this was also dropped at this stage of the project. To summarise, only two threshold values have been considered from this point: less than 0.1 mm and less than 1.0 mm.

Analysis of basic statistics of the threshold values yielded the data shown in Table ?!?!? (_see the table at C:\Users\b1043453\OneDrive - Newcastle University\OnePlanet PhD\Random_mixing\RMWSPy_Horning_and_Haese_2021\RMWSPy-master\MCRB_examples\MCRB_gauges_only\Characterising_P_statistics\Prob_zero_P_stats_2005-2016\Prob_zero_P_stats_2005-2016.csv)_. The table shows that the mean, minimum and maximum values for thresholds of 0 mm, < 0.1 mm and <= to 0.1 mm are very similar. Only when considering a dry day threshold of 1.0 mm is there a significant difference in values, for example mean probability values range from 0.54 - 0.55 for the three lower thresholds, to 0.68 with a 1.0 mm threshold. **Dry day thresholds are discussed further in Section ?!?!?!?!?**

In [2]:
import numpy as np
import pandas as pd
import math
import scipy as st
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline
from numpy import genfromtxt

### Read in P data for all 8 stations from csv file created in P_inputs_allstns_2005-2016.ipynb

In [3]:
# Use pd.read_csv to read csv file
allstnsP_df = pd.read_csv(r"C:\Users\b1043453\OneDrive - Newcastle University\OnePlanet PhD\Random_mixing\RMWSPy_Horning_and_Haese_2021\RMWSPy-master\MCRB_examples\MCRB_gauges_only\Characterising_P_statistics\Pinputs_20km_stns.csv",
                         index_col=0)
allstnsP_df.head()

,FR_p_mm,UC_p_mm,HM_p_mm,K_p_mm,BV_p_mm
date,,,,,
2005-10-01,16.0160,11.3028,7.3722,0.0,10.4
2005-10-02,0.0000,0.0000,0.0000,0.4,0.0
2005-10-03,0.7649,0.5399,0.0000,1.2,0.2
2005-10-04,0.5463,0.3855,0.0000,0.0,0.4
2005-10-05,0.0896,0.0633,0.0000,0.0,0.0


### Calculate probability of zero P (threshold < 0.1 mm) for each station using a function which will feed into run_inv.py

In [4]:
def calcP0(x):
    result_p0 = (x < 0.1).sum() / len(x)
    return result_p0

test = allstnsP_df.apply(calcP0, axis=0)
test.values

array([0.44190097, 0.52102513, 0.61408311, 0.67355063, 0.57327693])

In [5]:
# # round P values to 1 dp for threshold calculations
# allstnsP1dp_df = allstnsP_df.round(1)
# allstnsP1dp_df.head()

### Probability of zero P: threshold = 0mm

In [6]:
# Calculate proportion of dry days: divide no of zeros in p_mm column by the length of the dataframe
FRprob_0 = (allstnsP_df.FR_p_mm == 0).sum() / len(allstnsP_df)
UCprob_0 = (allstnsP_df.UC_p_mm == 0).sum() / len(allstnsP_df)
HMprob_0 = (allstnsP_df.HM_p_mm == 0).sum() / len(allstnsP_df)
Kprob_0 = (allstnsP_df.K_p_mm == 0).sum() / len(allstnsP_df)
BVprob_0 = (allstnsP_df.BV_p_mm == 0).sum() / len(allstnsP_df)

### Probability of zero P: threshold < 0.1mm

In [7]:
# Calculate proportion of dry days: divide no of < 0.1mm values in p_mm column by the length of the dataframe
FRprob_less_0pt1 = (allstnsP_df.FR_p_mm < 0.1).sum() / len(allstnsP_df)
UCprob_less_0pt1 = (allstnsP_df.UC_p_mm < 0.1).sum() / len(allstnsP_df)
HMprob_less_0pt1 = (allstnsP_df.HM_p_mm < 0.1).sum() / len(allstnsP_df)
Kprob_less_0pt1 = (allstnsP_df.K_p_mm < 0.1).sum() / len(allstnsP_df)
BVprob_less_0pt1 = (allstnsP_df.BV_p_mm < 0.1).sum() / len(allstnsP_df)

### Probability of zero P: threshold <= 0.1mm

In [8]:
# Calculate proportion of dry days: divide no of ,= 0.1 mm values in p_mm column by the length of the dataframe
FRprob_lessequal_0pt1 = (allstnsP_df.FR_p_mm <= 0.1).sum() / len(allstnsP_df)
UCprob_lessequal_0pt1 = (allstnsP_df.UC_p_mm <= 0.1).sum() / len(allstnsP_df)
HMprob_lessequal_0pt1 = (allstnsP_df.HM_p_mm <= 0.1).sum() / len(allstnsP_df)
Kprob_lessequal_0pt1 = (allstnsP_df.K_p_mm <= 0.1).sum() / len(allstnsP_df)
BVprob_lessequal_0pt1 = (allstnsP_df.BV_p_mm <= 0.1).sum() / len(allstnsP_df)

### Probability of zero P: threshold < 1.0mm

In [9]:
# Calculate proportion of dry days: divide no of < 1.0 mm values in p_mm column by the length of the dataframe
FRprob_less_1 = (allstnsP_df.FR_p_mm < 1.0).sum() / len(allstnsP_df)
UCprob_less_1 = (allstnsP_df.UC_p_mm < 1.0).sum() / len(allstnsP_df)
HMprob_less_1 = (allstnsP_df.HM_p_mm < 1.0).sum() / len(allstnsP_df)
Kprob_less_1 = (allstnsP_df.K_p_mm < 1.0).sum() / len(allstnsP_df)
BVprob_less_1 = (allstnsP_df.BV_p_mm < 1.0).sum() / len(allstnsP_df)

## Create a dataframe in order of elevation (highest to lowest)

In [10]:
# step 1: initialise my lists
data = [["FR_2325m", FRprob_0, FRprob_less_0pt1, FRprob_lessequal_0pt1, FRprob_less_1],
       ["UC_1845m", UCprob_0, UCprob_less_0pt1, UCprob_lessequal_0pt1, UCprob_less_1],
       ["HM_1436m", HMprob_0, HMprob_less_0pt1, HMprob_lessequal_0pt1, HMprob_less_1],
       ["K_1391m", Kprob_0, Kprob_less_0pt1, Kprob_lessequal_0pt1, Kprob_less_1],
       ["BV_1298m", BVprob_0, BVprob_less_0pt1, BVprob_lessequal_0pt1, BVprob_less_1]]
# data

In [11]:
# step 2: create df from lists
prob_df = pd.DataFrame(data, columns = ["station", "prob_0mm", "prob<0.1mm", "prob<=0.1mm", "prob<1mm"])
prob_df.set_index("station", inplace=True)
prob_df

,prob_0mm,prob<0.1mm,prob<=0.1mm,prob<1mm
station,,,,
FR_2325m,0.420254,0.441901,0.441901,0.631252
UC_1845m,0.468027,0.521025,0.521025,0.719084
HM_1436m,0.575267,0.614083,0.614083,0.772083
K_1391m,0.673551,0.673551,0.673799,0.740980
BV_1298m,0.573277,0.573277,0.573277,0.721573


### Write dataframe to csv

In [14]:
prob_df.to_csv(r"C:\Users\b1043453\OneDrive - Newcastle University\OnePlanet PhD\Random_mixing\RMWSPy_Horning_and_Haese_2021\RMWSPy-master\MCRB_examples\MCRB_gauges_only\Characterising_P_statistics\Prob_zero_P_20km_stns.csv")

In [15]:
prob_df.describe()

,prob_0mm,prob<0.1mm,prob<=0.1mm,prob<1mm
count,5.000000,5.000000,5.000000,5.000000
mean,0.542075,0.564767,0.564817,0.716994
std,0.099605,0.088527,0.088603,0.052410
min,0.420254,0.441901,0.441901,0.631252
25%,0.468027,0.521025,0.521025,0.719084
50%,0.573277,0.573277,0.573277,0.721573
75%,0.575267,0.614083,0.614083,0.740980
max,0.673551,0.673551,0.673799,0.772083
